## Imports

In [ ]:
# from preprocessing import docx_processing  as doc, textract_processing as txt
# from text_processing import tf_idf_cosine_similarity as tf_idf,doc2vec_comparison as d2v
# from text_processing import cv_cosine_similarity as cv
# import os


## Pre-Processing

In [32]:
import os
import textract
import re

def get_content_as_string(filename):
    """
    use textract library to extract text from varied file formats like (.docx, .doc, .gif, .csv, .pdf, .png, ...etc) using a single api

    :params:
    filename : name of file to be extracted to text

    :returns:
    lower_case_string : text content of data in lowercase
    """
    text = textract.process(filename)
    lower_case_string =  str(text.decode('utf-8')).lower()
    #final_string = re.sub('[^a-zA-Z0-9 \n]', '', lower_case_string)
    return lower_case_string

In [39]:
from nltk.corpus import stopwords
# Caution when deploying
# TODO(done) : Download stopwords once initially
# import nltk
# nltk.download('stopwords')

english_stopwords = set(stopwords.words('english'))

In [40]:
#  remove stop words and lemmatize
from nltk.stem.wordnet import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords as stp
# from preprocessing import lemma_tagger as tag

import nltk
from nltk.corpus import wordnet

def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)
lemmatizer = WordNetLemmatizer()
analyzer = TfidfVectorizer().build_analyzer()


def stemmed_words(doc):
    # return (lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in analyzer(doc) if w not in set(stp.words('english')))
    return (lemmatizer.lemmatize(w,get_wordnet_pos(w)) for w in analyzer(doc) if w not in english_stopwords)


## Processing

### Text Processing

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.wordnet import WordNetLemmatizer
# from nltk.corpus import stopwords as stp


def get_tf_idf_cosine_similarity(compare_doc,doc_corpus):
    # lemmatizer = WordNetLemmatizer()
    # analyzer = TfidfVectorizer().build_analyzer()
    #
    # def stemmed_words(doc):
    #     return (lemmatizer.lemmatize(w) for w in analyzer(doc) if w not in set(stp.words('english')))

    tf_idf_vect = TfidfVectorizer(analyzer=stemmed_words)
    #tf_idf_vect = TfidfVectorizer(stop_words='english')
    tf_idf_req_vector = tf_idf_vect.fit_transform([compare_doc]).todense()
    #tf_idf_req_vector = tf_idf_vect.fit_transform(doc_corpus).todense()
    #print('Features are:', len(tf_idf_vect.get_feature_names()))
    #print(tf_idf_vect.get_feature_names())
    tf_idf_resume_vector = tf_idf_vect.transform(doc_corpus).todense()
    #tf_idf_resume_vector = tf_idf_vect.transform([compare_doc]).todense()
    cosine_similarity_list = []
    for i in range(len(tf_idf_resume_vector)):
        cosine_similarity_list.append(cosine_similarity(tf_idf_req_vector,tf_idf_resume_vector[i])[0][0])
    # for i in range(len(tf_idf_req_vector)):
    #   cosine_similarity_list.append(cosine_similarity(tf_idf_resume_vector,tf_idf_req_vector[i])[0][0])
    return cosine_similarity_list

In [35]:
def process_files(job_desc,resume_list):

    job_desc_text = get_content_as_string(job_desc)
    resume_list_text = []
    for cur_resume in resume_list:
        resume_list_text.append(get_content_as_string(cur_resume))

    #similarity_list = d2v.get_doc2vec_similarity(req_doc_text, resume_doc_text)
    #cos_sim_list = tf_idf.get_tf_cosine_similarity(req_doc_text,resume_doc_text)
    # zipped_docs = zip(cos_sim_list,resume_docs)
    # sorted_doc_list = sorted(zipped_docs, key = lambda x: x[0],reverse=True)
    # final_doc_rating_list = []
    # for element in similarity_list:
    #     doc_rating_list = []
    #     doc_rating_list.append(os.path.basename(resume_docs[element[0]]))
    #     doc_rating_list.append("{:.0%}".format(element[1]))
    #     final_doc_rating_list.append(doc_rating_list)
    #print(final_doc_rating_list)
    #return final_doc_rating_list
    # print('TF cosine similarity')
    #print(final_doc_rating_list)



    # TF-IDF - cosine similarity
    # final_doc_rating_list = []
    cos_sim_list = get_tf_idf_cosine_similarity(job_desc_text,resume_list_text)
    final_doc_rating_list = []
    zipped_docs = zip(cos_sim_list,resume_list)
    sorted_doc_list = sorted(zipped_docs, key = lambda x: x[0], reverse=True)
    for element in sorted_doc_list:
        doc_rating_list = []
        doc_rating_list.append(os.path.basename(element[1]))
        doc_rating_list.append("{:.0%}".format(element[0]))
        final_doc_rating_list.append(doc_rating_list)
    return final_doc_rating_list
    # print('TF-IDF cosine similarity')
    # print(final_doc_rating_list)
    #
    # # binary cv - cosine similarity
    # final_doc_rating_list = []
    # cos_sim_list = cv.get_binay_cosine_similarity(req_doc_text,resume_doc_text)
    #
    # zipped_docs = zip(cos_sim_list,resume_docs)
    # sorted_doc_list = sorted(zipped_docs, key = lambda x: x[0],reverse=True)
    # for element in sorted_doc_list:
    #     doc_rating_list = []
    #     doc_rating_list.append(os.path.basename(element[1]))
    #     doc_rating_list.append("{:.0%}".format(element[0]))
    #     final_doc_rating_list.append(doc_rating_list)
    # print('Binary BOW cosine similarity')
    # print(final_doc_rating_list)
    #
    # # Regular cv - cosine similarity
    # final_doc_rating_list = []
    # cos_sim_list = cv.get_cosine_similarity(req_doc_text,resume_doc_text)
    #
    # zipped_docs = zip(cos_sim_list,resume_docs)
    # sorted_doc_list = sorted(zipped_docs, key = lambda x: x[0],reverse=True)
    # for element in sorted_doc_list:
    #     doc_rating_list = []
    #     doc_rating_list.append(os.path.basename(element[1]))
    #     doc_rating_list.append("{:.0%}".format(element[0]))
    #     final_doc_rating_list.append(doc_rating_list)
    # print(' BOW cosine similarity')
    # print(final_doc_rating_list)


## Code Runner

In [43]:
if __name__ == "__main__":

    req_document = r'C:\Users\UNNAT\UNNAT_CLG\Project_Resume_Ranking\resume_ranker\samples\job_description_samples\amazon_data_scientist_ii.txt'
    resume_docs = [
        # r'C:\Users\UNNAT\UNNAT_CLG\Project_Resume_Ranking\resume_ranker\samples\resume_samples\Unnat_Resume.pdf',
        r'C:\Users\UNNAT\UNNAT_CLG\Project_Resume_Ranking\resume_ranker\samples\resume_samples\backend_developer_resume_amazon.png',
        r'C:\Users\UNNAT\UNNAT_CLG\Project_Resume_Ranking\resume_ranker\samples\resume_samples\infra_sec_resume.png'
        # r'C:\Users\UNNAT\UNNAT_CLG\Project_Resume_Ranking\resume_ranker\samples\resume_samples\data_scientist_resume_example.pdf'
    ]

    result_ranking = process_files(req_document,resume_docs)
    print(result_ranking)
    # lemmatizer = WordNetLemmatizer()
    # word_list = ['lead', 'leadership', 'leading']
    # op = ' '.join(lemmatizer.lemmatize(w) for w in word_list)
    # print(op)

    # text = textract.process(r"C:\Users\UNNAT\UNNAT_CLG\Project_Resume_Ranking\resume_ranker\samples\resume_samples\Unnat_Resume.pdf", method="pdfminer").decode("utf8")
    # print(text)
       
    print("------------------------")
    print("Current Job Description")
    print("------------------------")
    print(get_content_as_string(req_document))
    
    print("------------------------")
    print("Current Resumes")
    print("------------------------")
    for ind,resume in enumerate(resume_docs):
        print(f"Resume {ind}\n------------------------")
        print(get_content_as_string(resume))

    # TODO : issue with converting pdf to text using textract (fix it) 
    # https://stackoverflow.com/questions/63359767/textract-failed-with-exit-code-127-pdftotext-on-windows-10 


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - 'C:\\Users\\UNNAT/nltk_data'
    - 'c:\\Users\\UNNAT\\.conda\\envs\\resume_ranker_env\\nltk_data'
    - 'c:\\Users\\UNNAT\\.conda\\envs\\resume_ranker_env\\share\\nltk_data'
    - 'c:\\Users\\UNNAT\\.conda\\envs\\resume_ranker_env\\lib\\nltk_data'
    - 'C:\\Users\\UNNAT\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
